In [24]:
import os
import cv2
import pandas as pd
import numpy as np

## Load data

In [25]:
label_colors = [    
    [128, 64, 128], [244, 35, 232], [70, 70, 70], [102, 102, 156], 
    [190, 153, 153], [153, 153, 153], [250, 170, 30], [220, 220, 0], 
    [107, 142, 35], [152, 251, 152], [70, 130, 180], [220, 20, 60], 
    [255, 0, 0], [0, 0, 142], [0, 0, 70], [0, 60, 100], [0, 80, 100], 
    [0, 0, 230], [119, 11, 32], [0, 0, 0]
]

label_names = [    
    'road', 'sidewalk', 'building', 'wall', 'fence', 'pole', 'traffic light',     
    'traffic sign', 'vegetation', 'terrain', 'sky', 'person', 'rider', 'car', 
    'truck', 'bus', 'train', 'motorcycle', 'bicycle', 'unlabeled']

label_dict = {label_names[i]: label_colors[i] for i in range(len(label_names))}

In [26]:
data_dir = 'gtFine/'
path_to_images = None
for dir in ['train', 'val']:
    split_dir = data_dir + dir + '/'
    path_to_images = [f'{split_dir}{f}' for f in \
         os.listdir(split_dir) if f.endswith('_gtFine_color.png')]

In [27]:
X = []
y = []
for path in  path_to_images:
    image = cv2.imread(path)
    labels = np.unique(image.reshape(-1, 3), axis=0)
    intersection = list(set(map(tuple, labels)).intersection(set(map(tuple, label_colors))))
    intersection = [list(item) for item in intersection]
    _label = []
    for index in range(len(label_colors)):
        if (label_colors[index] in intersection):
            _label.append(1)
        else: _label.append(0)
    X.append(image)
    y.append(_label)

In [28]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, jaccard_score, \
                    precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [29]:
# Convert the lists to numpy arrays
X = np.array(X)
X = np.reshape(X, (X.shape[0], X.shape[1] * X.shape[2] * X.shape[3]))
y = np.array(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Model

In [30]:
logreg = LogisticRegression(multi_class='multinomial', solver='lbfgs', max_iter=1000, random_state=42)

## One-vs-all

In [50]:
perfs = []
for index_ in range(len(label_colors)):
    try:
        list_ = []
        logreg.fit(X_train, y_train[:, index_])
        
        # Evaluate the SVM model
        y_pred = logreg.predict(X_test)
        
        list_.append(label_names[index_])
        list_.append(jaccard_score(y_test[:, index_], y_pred, average='macro'))
        list_.append(accuracy_score(y_test[:, index_], y_pred))
        list_.append(precision_score(y_test[:, index_], y_pred))
        list_.append(recall_score(y_test[:, index_], y_pred, average='weighted'))
        list_.append(f1_score(y_test[:, index_], y_pred, average='weighted'))

        perfs.append(list_)
    
    except ValueError:
        perfs.append([label_names[index_], 0.2,0.2,0.2,0.2,0.2])

/home/richardn/Documents/IDU/AIBiaisHunter/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/richardn/Documents/IDU/AIBiaisHunter/env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [51]:
df = pd.DataFrame(perfs, columns=['class', 'jaccard_score', 'accuracy_score', 'precision_score', 'recall_score', 'f1_score'])

In [52]:
df.to_csv('output.csv')